In [1]:
import torch
import random
import datetime
import pandas as pd
import numpy as np
import os

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, fit_treshold
from src.embeddings import RecsysEmbedding
from sklearn.linear_model import LogisticRegression

experiment_name = 'SlatewiseGRU'
device = 'cuda:2'
seed = 7331
pkl_path = '../pkl/'


random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
def flatten(true, pred, mask, to_cpu=True):
    mask = mask.flatten()
    nnz_idx = mask.nonzero()[:, 0]
    true, pred = [x.flatten()[nnz_idx] for x in [true, pred]]
    if to_cpu:
        true, pred = [x.cpu().numpy() for x in [true, pred]]
    return true, pred

# Модель

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class SlatewiseGRU(nn.Module):
    def __init__(self, embedding, readout=False):
        super().__init__()
        
        self.embedding = embedding
        
        self.emb_dim = embedding.embedding_dim
        self.rnn_layer = nn.GRU(
            input_size=self.emb_dim * 2, 
            hidden_size=self.emb_dim, 
            batch_first=True
        )
        self.out_layer = nn.Linear(self.emb_dim, 1)
        
        self.thr = -1.5
        self.readout = readout
        self.readout_mode = 'threshold' # ['soft' ,'threshold', 'sample', 'diff_sample']
        
        self.calibration = False
        self.w = 1
        self.b = 0

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        max_sequence = item_embs.size(1)
        # ilya format:
        # 'items': (batch, slate, 2*embedding_dim ) 2, нужно для readout, по умолчанию ноли на половине эмбеддинга
        # 'clicks': (batch, slate)
        # 'users': (1, batch, embedding_dim), 
        # 'mask': (batch, slate)
        
        x = {}
        x['items'] = torch.cat(
            [
                item_embs.flatten(0,1),
                torch.zeros_like(item_embs.flatten(0,1)),
            ],
            dim = -1
        )
                
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2).unsqueeze(0)
        x['users'] = user_embs.repeat_interleave(max_sequence, 1)
        x['clicks'] = (batch['responses'].flatten(0,1) > 0 ).int().clone()
        x['mask'] = batch['slates_mask'].flatten(0,1).clone()
        
        items = x['items']
        h = x['users']
        
        if self.readout:
            res = []
            seq_len = items.shape[1]
            for i in range(seq_len):
#                 print(items[:,[i],:])
                output, h = self.rnn_layer(items[:,[i],:], h)
                y = self.out_layer(output)[:, :, 0]
                
                # readout
                if i + 1 < seq_len:
                    if self.readout_mode == 'threshold':
                        items[:, [i+1], self.emb_dim:] *= (y.detach()[:, :, None] > self.thr).to(torch.float32)
                    elif self.readout_mode == 'soft':
                        items[:, [i+1], self.emb_dim:] *= torch.sigmoid(y)[:, :, None]
                    elif self.readout_mode == 'diff_sample' or self.readout_mode == 'sample':
                        eps = 1e-8
                        gumbel_sample = -( (torch.rand_like(y) + eps).log() / (torch.rand_like(y) + eps).log() + eps).log()
                        T = 0.3
                        bernoulli_sample = torch.sigmoid( (nn.LogSigmoid()(self.w * y + self.b) + gumbel_sample) / T )
                        if self.readout_mode == 'sample':
                            bernoulli_sample = bernoulli_sample.detach()
                        items[:, [i+1], self.emb_dim:] *= bernoulli_sample[:, :, None]
                    else:
                        raise
                    
                res.append(y)
        
            y = torch.cat(res, axis=1)
            
        else:
            items[:, 1:, self.emb_dim:] *= x['clicks'][:, :-1, None]
            rnn_out, _ = self.rnn_layer(items, h)
            y = self.out_layer(rnn_out)[:, :, 0]
        
        
        if self.calibration and self.training:
            clicks_flat, logits_flat = flatten(x['clicks'], y.detach(), x['mask'])
            logreg = LogisticRegression()
            logreg.fit(logits_flat[:, None], clicks_flat)
            γ = 0.3
            self.w = (1 - γ) * self.w + γ * logreg.coef_[0, 0]
            self.b = (1 - γ) * self.b + γ * logreg.intercept_[0]
            y = self.w * y + self.b
        else:
            y = self.w * y + self.b
            
        return y.reshape(shp[:-1])

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SlatewiseGRU(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='svd', embedding_dim = 2),
    readout=False
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 3458.74it/s]


Test before learning: {'f1': 0.0, 'roc-auc': 0.3333333134651184, 'accuracy': 0.75}


train... loss:0.7181857228279114:   0%|                                                                                                    | 1/5000 [00:00<26:43,  3.12it/s]

Val update: epoch: 0 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.3333333134651184 | treshold: 0.01
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.3333333134651184 | 


...

train... loss:0.6820520758628845:   2%|█▋                                                                                                 | 85/5000 [00:19<20:37,  3.97it/s]

Val update: epoch: 84 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.36000000000000004
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 


train... loss:0.6675306558609009:   2%|██▎                                                                                               | 117/5000 [00:26<18:25,  4.42it/s]

Val update: epoch: 117 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.38
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SlatewiseGRU(
   (embedding): RecsysEmbedding()
   (rnn_layer): GRU(4, 2, batch_first=True)
   (out_layer): Linear(in_features=2, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = SlatewiseGRU(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings).to('cpu'),
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating SlatewiseGRU with svd embeddings
Test before learning: {'f1': 0.17239901423454285, 'roc-auc': 0.4641438126564026, 'accuracy': 0.11004862934350967}


train... loss:49.18179851770401:   0%|                                                                                                  | 1/5000 [00:39<55:26:15, 39.92s/it]

Val update: epoch: 0 |accuracy: 0.1531217247247696 | f1: 0.18792562186717987 | auc: 0.6076698899269104 | treshold: 0.09
Test: accuracy: 0.17369838058948517 | f1: 0.1841088980436325 | auc: 0.6171619892120361 | 


train... loss:34.75427806377411:   0%|                                                                                                  | 2/5000 [00:47<29:18:44, 21.11s/it]

Val update: epoch: 1 |accuracy: 0.7042703628540039 | f1: 0.23013544082641602 | auc: 0.608708381652832 | treshold: 0.13
Test: accuracy: 0.7140371203422546 | f1: 0.2277820110321045 | auc: 0.6147950887680054 | 


...

train... loss:27.960309609770775:   0%|▏                                                                                               | 11/5000 [01:48<13:36:36,  9.82s/it]

Val update: epoch: 10 |accuracy: 0.8797783255577087 | f1: 0.29637661576271057 | auc: 0.7294415235519409 | treshold: 0.19
Test: accuracy: 0.8848801851272583 | f1: 0.2837049961090088 | auc: 0.7288740873336792 | 


train... loss:26.77064834535122:   0%|▎                                                                                                | 16/5000 [02:36<13:28:21,  9.73s/it]

Val update: epoch: 15 |accuracy: 0.8843481540679932 | f1: 0.2902889549732208 | auc: 0.730168342590332 | treshold: 0.22
Test: accuracy: 0.8881370425224304 | f1: 0.28005358576774597 | auc: 0.7283613681793213 | 


train... loss:25.189408838748932:   1%|▋                                                                                               | 37/5000 [06:05<13:37:07,  9.88s/it]


In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SlatewiseGRU(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating SlatewiseGRU with neural embeddings
Test before learning: {'f1': 0.16593550145626068, 'roc-auc': 0.48787927627563477, 'accuracy': 0.3701607882976532}


train... loss:63.73415842652321:   0%|                                                                                                  | 1/5000 [01:03<87:35:14, 63.08s/it]

Val update: epoch: 0 |accuracy: 0.7681758999824524 | f1: 0.8368760347366333 | auc: 0.8075522184371948 | treshold: 0.39
Test: accuracy: 0.7684923410415649 | f1: 0.8374698758125305 | auc: 0.8102037906646729 | 


train... loss:48.87808521091938:   0%|                                                                                                  | 2/5000 [01:09<41:34:08, 29.94s/it]

Val update: epoch: 1 |accuracy: 0.8034634590148926 | f1: 0.8563498854637146 | auc: 0.8670310974121094 | treshold: 0.41000000000000003
Test: accuracy: 0.7996131181716919 | f1: 0.8537545204162598 | auc: 0.8656007051467896 | 


...

train... loss:27.834611371159554:   0%|▎                                                                                                | 17/5000 [02:47<9:05:09,  6.56s/it]

Val update: epoch: 16 |accuracy: 0.8505780696868896 | f1: 0.8832356929779053 | auc: 0.919102668762207 | treshold: 0.37
Test: accuracy: 0.8501511216163635 | f1: 0.8826969861984253 | auc: 0.9197525978088379 | 


train... loss:25.706352680921555:   1%|▋                                                                                                | 36/5000 [04:47<8:39:01,  6.27s/it]/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
train... loss:22.061962768435478:   2%|█▉                                                                                              | 101/5000 [11:40<9:26:08,  6.93s/it]



Evaluating SlatewiseGRU with explicit embeddings


RuntimeError: input.size(-1) must be equal to input_size. Expected 64, got 80

In [ ]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items